In [1]:
import socket

#socket.AF_INET => Using IPv4
#socket.SOCK_STREAM => Using TCP
listen_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#'127.0.0.1 loop back address, 2345 port number
listen_socket.bind(('127.0.0.1', 12345))
print("waiting connection...")
listen_socket.listen()
game_socket, addr = listen_socket.accept()
print("connection accepted:")


store_list = {"apple":0.9,
            "banana":1.2,
            "carrot":1.5,
            "dragon fruit":3.0,
            "egg plant":2.6}

done = False
while not done:
    # server always waits for client's msg
    rx_msg_str = ""
    while "\n" not in rx_msg_str:
        rx_msg_str += game_socket.recv(1024).decode()
        
    if rx_msg_str[:5] == "CHECK":
        msg_check = "CHECK"
        header, item_name = rx_msg_str[:-1].split(',')
        if item_name in store_list:
            msg_check += "," + item_name + "," + str(store_list[item_name])
        msg_check += "\n"
        game_socket.send(msg_check.encode())

            
    elif rx_msg_str[:6] == "UPDATE":
        msg_update = "UPDATE"
        header, item_name, price = rx_msg_str[:-1].split(',')
        if item_name in store_list:
            store_list[item_name] = float(price)
            msg_update += ",OK"
        else:
            msg_update += ",FAIL"
        msg_update += "\n"
        game_socket.send(msg_update.encode())
            
    elif rx_msg_str[:6] == "DELETE":
        msg_delete = "DELETE"
        header, item_name = rx_msg_str[:-1].split(',')
        if item_name in store_list:
            store_list.pop(item_name)
            msg_delete += ",OK"
        else:
            msg_delete += ",FAIL"
        msg_delete += "\n"
        game_socket.send(msg_delete.encode())
            
    elif rx_msg_str[:-1] == "QUIT":
        done = True
        game_socket.send("QUIT\n".encode())
        print("User quits.")
    else:
        raise "Invalid command."
        
game_socket.close()
listen_socket.close()

waiting connection...
connection accepted:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host